In [ ]:
import numpy as np
import pandas as pd

In [ ]:
coinbase = np.load('/content/coinbase_order_array.npy', allow_pickle=True)
coinbase

array([{'bids': [['0.02706', '0.3941605', 2]], 'asks': [['0.02707', '17.115', 2]], 'sequence': 2517907125, 'time': 1609195957834},
       {'bids': [['0.02706', '0.1191605', 1]], 'asks': [['0.02707', '3.075', 3]], 'sequence': 2517907249, 'time': 1609195959133},
       {'bids': [['0.02707', '2.8', 1]], 'asks': [['0.02708', '24.03321021', 4]], 'sequence': 2517907434, 'time': 1609195960437},
       ...,
       {'bids': [['0.0273', '11.82427217', 3]], 'asks': [['0.02731', '5.87095917', 2]], 'sequence': 2519953418, 'time': 1609239371302},
       {'bids': [['0.02729', '24.01996707', 7]], 'asks': [['0.0273', '5.87095917', 2]], 'sequence': 2519953656, 'time': 1609239373100},
       {'bids': [['0.02729', '29.53056707', 7]], 'asks': [['0.0273', '5.87095917', 2]], 'sequence': 2519953695, 'time': 1609239374755}],
      dtype=object)

In [ ]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 63kB/s 
     |████████████████████████████████| 204kB 43.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=aa63b83a4b7976c14c64640b2bb7fe4e36fabc31c56b8ebc9cd3a5d6c14bcde1
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.ml.linalg import Vectors
import pyspark
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
sc = spark.sparkContext
ssc = StreamingContext(sc, 10)
rdd = ssc.sparkContext.parallelize(coinbase)

In [ ]:
rdd = rdd.map(lambda x: (x['time'], x['bids'], x['asks']))

In [ ]:
rdd.take(5)

[(1609195957834, [['0.02706', '0.3941605', 2]], [['0.02707', '17.115', 2]]),
 (1609195959133, [['0.02706', '0.1191605', 1]], [['0.02707', '3.075', 3]]),
 (1609195960437, [['0.02707', '2.8', 1]], [['0.02708', '24.03321021', 4]]),
 (1609195962189,
  [['0.02708', '4.84406447', 3]],
  [['0.02709', '22.27755749', 2]]),
 (1609195963505,
  [['0.02708', '4.84406447', 3]],
  [['0.02709', '22.27755749', 2]])]

In [ ]:
def min_max(iterator):

   
    asks=[]
    for x in iterator:

      asks.append([x[0],x[1][0][0], x[2][0][0]])

    return asks

In [ ]:
final = rdd.mapPartitions(min_max)

In [ ]:
final.take(10)

[[1609195957834, '0.02706', '0.02707'],
 [1609195959133, '0.02706', '0.02707'],
 [1609195960437, '0.02707', '0.02708'],
 [1609195962189, '0.02708', '0.02709'],
 [1609195963505, '0.02708', '0.02709'],
 [1609195964789, '0.02709', '0.0271'],
 [1609195966189, '0.02709', '0.02711'],
 [1609195967508, '0.02709', '0.02711'],
 [1609195969246, '0.02709', '0.02711'],
 [1609195970621, '0.0271', '0.02711']]

In [ ]:
bin_order = np.load('/content/binance_order_array.npy',allow_pickle=True)

In [ ]:
binance_order = ssc.sparkContext.parallelize (
    bin_order
)

In [83]:
merged = (binance_order.map(lambda x: (str(x['time'])[:-3],0)))\
.leftOuterJoin(
    final.map(lambda x: ((str(x[0])[:-3]), (x[1], x[2])))
    )

In [84]:
merged.take(100)

[('1609195964', (0, ('0.02709', '0.0271'))),
 ('1609195968', (0, None)),
 ('1609195970', (0, ('0.0271', '0.02711'))),
 ('1609195987', (0, ('0.02709', '0.0271'))),
 ('1609195990', (0, ('0.02706', '0.02708'))),
 ('1609195992', (0, None)),
 ('1609195993', (0, ('0.02706', '0.02709'))),
 ('1609195995', (0, ('0.02706', '0.02708'))),
 ('1609195996', (0, None)),
 ('1609196000', (0, None)),
 ('1609196001', (0, ('0.02706', '0.02707'))),
 ('1609196017', (0, ('0.02705', '0.02707'))),
 ('1609196020', (0, ('0.02706', '0.02708'))),
 ('1609196021', (0, ('0.02705', '0.02706'))),
 ('1609196023', (0, ('0.02705', '0.02706'))),
 ('1609196032', (0, ('0.02706', '0.02707'))),
 ('1609196037', (0, None)),
 ('1609196046', (0, ('0.02704', '0.02706'))),
 ('1609196061', (0, ('0.02702', '0.02705'))),
 ('1609196069', (0, ('0.02705', '0.02708'))),
 ('1609196071', (0, ('0.02705', '0.02708'))),
 ('1609196077', (0, None)),
 ('1609196082', (0, ('0.02706', '0.02708'))),
 ('1609196085', (0, ('0.02706', '0.02708'))),
 ('1609

In [85]:
merged = merged.map(lambda x: [x[0],x[1][1]])

In [86]:
merged.take(5)

[['1609195964', ('0.02709', '0.0271')],
 ['1609195968', None],
 ['1609195970', ('0.0271', '0.02711')],
 ['1609195987', ('0.02709', '0.0271')],
 ['1609195990', ('0.02706', '0.02708')]]

In [89]:
def arrangeNone(iterator):

    sv = 0

    out = []

    for x in iterator:
        if x[1] is None:
            x[1] = sv
        else:
            sv = x[1]

        out.append([x[0],x[1]])
        
    return out 

In [90]:
cb_order = merged.mapPartitions(arrangeNone)

In [94]:
cb_order.take(5)

[['1609195964', ('0.02709', '0.0271')],
 ['1609195968', ('0.02709', '0.0271')],
 ['1609195970', ('0.0271', '0.02711')],
 ['1609195987', ('0.02709', '0.0271')],
 ['1609195990', ('0.02706', '0.02708')]]

In [95]:
!pip install influxdb

     |████████████████████████████████| 81kB 3.6MB/s 


In [96]:
!sudo apt-get update && sudo apt-get install influxdb

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [41.5 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [98]:
!sudo service influxdb start

 * Starting database influxd
   ...done.


In [99]:
from influxdb import InfluxDBClient
client = InfluxDBClient(host='localhost', port=8086)

In [100]:
client.create_database('coinbase_order')

In [105]:
def influx(partitions):
    result = []
    for i in range(np.array(partitions).shape[0] - 1):
        result.append('coinbase_trade date='+partitions[i][0]+','+'bids='+partitions[i][1][0]+','+'asks='+ partitions[i][1][1])
    return result

In [106]:
cb_order=cb_order.mapPartitions(influx)  

In [107]:
cb_order.take(5)

['coinbase_trade date=1609195964,bids=0.02709,asks=0.0271',
 'coinbase_trade date=1609195968,bids=0.02709,asks=0.0271',
 'coinbase_trade date=1609195970,bids=0.0271,asks=0.02711',
 'coinbase_trade date=1609195987,bids=0.02709,asks=0.0271',
 'coinbase_trade date=1609195990,bids=0.02706,asks=0.02708']